# [GALAXY] DeepCR Remover

<div class="alert alert-block alert-info">
    <b>Note:</b> This notebook should be run with the <span style="font-family: 'Ariel', monospace;">DCR</span> environment.
    DeepCR has some issues with how the software has been updated. Within your environment, it may be necessary to implement
    the <a href="https://github.com/ebertin/deepCR/commit/e42eb1677155dd4edc12c9f9a1ab638c6ad4f195">changes introduced
    by Emanual Bertin</a>.
</div>

I have had some trouble removing CRs with AstroScrappy so this is an attempt to remove CRs with DeepCR instead.

## Imports

In [ ]:
# Python Imports
import os
from pathlib import Path

# Astropy Colab Imports
from astropy.io import fits

# Other Astronomy Imports
from deepCR import deepCR

## Notebook Setup

In [ ]:
# Get the Notebook's Directory and ChangeDir to it
NOTEBOOK_DIR = Path(__file__).resolve().parent
os.chdir(NOTEBOOK_DIR)

In [ ]:
# Data Directory
DATA_DIR = NOTEBOOK_DIR / Path('mastDownload/HST')

# FLC Glob Pattern
FLC_GLOB = DATA_DIR.rglob('*[!crclean]_fl?.fits')

## Store File Name Data

In [ ]:
# Get the File Names and Sort them by filter
fileNameDict = {}
for fn in FLC_GLOB:

    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']

    # Store the Name using the filter as the dict key
    # Start the Empty List if Key does not exist
    if filt not in fileNameDict:
        fileNameDict[filt] = []
    fileNameDict[filt].append(fn)

## Remove CRs

### Remove F814W CRs

In [ ]:
# Create the Model
mdl = deepCR(
    mask='ACS-WFC-F814W',
    device='GPU'
)

In [ ]:
# Loop Through Files
for fn in fileNameDict['F814W']:

    # Out Name
    outName = fn.with_name(fn.name.replace('flc.fits', 'crclean_flc.fits'))

    # Open Image
    with fits.open(fn) as hduList:

        # Get Cleaned Arrays
        crmsk1, crarr1 = mdl.clean(
            hduList['SCI', 1].data, #threshold=0.4,
            binary=True,
            inpaint=True, n_jobs=1)
        crmsk2, crarr2 = mdl.clean(
            hduList['SCI', 2].data, #threshold=0.4,
            binary=True,
            inpaint=True, n_jobs=1)

        # Write the Output Data
        outList = hduList.copy()  # Copy the Original File
        outList[0].header.add_history('CRs removed with DeepCR')  # Update comment about processing
        outList[0].header.add_comment('Created by [AUTHOR], [INSTITUTION]')
        outList[0].header.add_comment('Created with pipeline at https://github.com/wwaldron/galred')
        outList['SCI', 1].data = crarr1.astype('float32')  # Change the old SCI image for CR Removed img
        outList['DQ', 1].data[crmsk1.astype(bool)] |= 2  # Mark DQ array as "Data replaced by fill"
        outList.insert(4, fits.ImageHDU(crmsk1.astype('uint8'), hduList['SCI', 1].header, 'MSK'))
        outList['SCI', 2].data = crarr2.astype('float32')
        outList['DQ', 2].data[crmsk1.astype(bool)] |= 2  # Mark DQ array as "Data replaced by fill"
        outList.insert(8, fits.ImageHDU(crmsk2.astype('uint8'), hduList['SCI', 2].header, 'MSK'))
        outList.writeto(outName, overwrite=True)

### Remove F475W CRs

In [ ]:
# Create the Model
mdl = deepCR(
    mask='ACS-WFC-F435W',
    device='GPU'
)

In [ ]:
# Loop Through Files
for fn in fileNameDict['F475W']:

    # Out Name
    outName = fn.with_name(fn.name.replace('flc.fits', 'crclean_flc.fits'))

    # Open Image
    with fits.open(fn) as hduList:

        # Get Cleaned Arrays
        crmsk1, crarr1 = mdl.clean(
            hduList['SCI', 1].data,
            binary=True,
            inpaint=True, n_jobs=1
        )
        crmsk2, crarr2 = mdl.clean(
            hduList['SCI', 2].data,
            binary=True,
            inpaint=True, n_jobs=1
        )

        # Write the Output Data
        outList = hduList.copy()  # Copy the Original File
        outList[0].header.add_history('CRs removed with DeepCR')  # Update comment about processing
        outList[0].header.add_comment('Created by [AUTHOR], [INSTITUTION]')
        outList[0].header.add_comment('Created with pipeline at https://github.com/wwaldron/galred')
        outList['SCI', 1].data = crarr1.astype('float32')  # Change the old SCI image for CR Removed img
        outList['DQ', 1].data[crmsk1.astype(bool)] |= 2  # Mark DQ array as "Data replaced by fill"
        outList.insert(4, fits.ImageHDU(crmsk1.astype('uint8'), hduList['SCI', 1].header, 'MSK'))
        outList['SCI', 2].data = crarr2.astype('float32')
        outList['DQ', 2].data[crmsk1.astype(bool)] |= 2  # Mark DQ array as "Data replaced by fill"
        outList.insert(8, fits.ImageHDU(crmsk2.astype('uint8'), hduList['SCI', 2].header, 'MSK'))
        outList.writeto(outName, overwrite=True)